In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

#load dataset
df = pd.read_csv('training_set_VU_DM.csv')

In [ ]:
# convert date to numerical feature 
df = df.sort_values(by="date_time")
df["date_time"] = (pd.to_datetime(df["date_time"]) - pd.Timestamp("1970-01-01")) // pd.Timedelta("1s")

In [ ]:
# drop columns with lot of nans and uncorrelated columns
drop_columns_nans = ['visitor_hist_starrating', 'visitor_hist_adr_usd','srch_query_affinity_score','comp1_rate', 'comp1_inv',
       'comp1_rate_percent_diff', 'comp2_rate', 'comp2_inv',
       'comp2_rate_percent_diff', 'comp3_rate', 'comp3_inv',
       'comp3_rate_percent_diff', 'comp4_rate', 'comp4_inv',
       'comp4_rate_percent_diff', 'comp5_rate', 'comp5_inv',
       'comp5_rate_percent_diff', 'comp6_rate', 'comp6_inv',
       'comp6_rate_percent_diff', 'comp7_rate', 'comp7_inv',
       'comp7_rate_percent_diff', 'comp8_rate']
drop_columns_low_correlation = ['site_id','prop_log_historical_price', 'srch_destination_id']
df = df.drop(drop_columns_nans, axis=1)
df = df.drop(drop_columns_low_correlation, axis=1)

In [ ]:
df_wo_outliers = df.copy(deep=True)
for index,p in enumerate(df['prop_id'].unique()):
#p=93974
    if index % 1000 == 0:
        print(index)
    new_df = df.loc[(df['prop_id'] == p)]
    outliers_p = new_df[np.abs(stats.zscore(new_df['price_usd']))>1].index
    df_wo_outliers.drop(outliers_p, axis=0)

hotel_means = df_wo_outliers.groupby('prop_id')['price_usd'].mean().reset_index()


df_merged = df.merge(hotel_means, on='prop_id', suffixes=('', '_mean'))

threshold = 0.3
def replace_with_mean(row):
    if row['price_usd'] < row['price_usd_mean'] * threshold:
        print(row['prop_id'])
        print(row['price_usd'], row['price_usd_mean'])
        return row['price_usd_mean']
    else:
        return row['price_usd']

df_merged['adjusted_price'] = df_merged.apply(replace_with_mean, axis=1)

df_merged = df_merged.drop('price_usd', axis=1).rename(columns={'adjusted_price': 'price_usd'})



In [ ]:
df.to_csv('cleaned.csv')